<!-- Auto table of contents -->
<h1 class='tocIgnore'>Naturalist : Extraction et exploitation des traces</h1>
<ul>
  <li>présentes dans les formulaires à partir Naturalist V0.128 (ou beta mai 2019),</li>
  <li>à condition de cocher la case "Enregistrer ma trace" en début de formulaire,</li>
  <li>via l'export Excel exclusivement
      (pas encore d'API pour ça, et absent des exports XML, JSON, KML, CSV en décembre 2019),</li>
  <li>avec la colonne "trace" sélectionnée dans l'export,</li>
  <li>uniquement via Faune-France (pas dispo. via les sites régionaux).</li>
</ul>
<p>Lecture XLSX et carto. publiée avec données anonymisées sur https://framagit.org/lpo/partage-de-codes le 25/01/2020</p>
<div style="overflow-y: auto">
  <h2 class='tocIgnore'>Table des matières</h2>
  <div id="toc"></div>
</div>

In [ ]:
%%javascript
$.getScript('ipython_notebook_toc.js')

In [ ]:
import sys
import os
import datetime as dt
import pandas as pd
import numpy as np

import folium
import folium.plugins

from collections import OrderedDict as odict
import json

from IPython.display import HTML

# Lecture des données exportées de Faune-France

(on ne garde que celles des formulaires)

In [ ]:
# ACDC 2019 : 2nd passage (fonctionnalité "trace" )
fn = f'tmp/ACDC2019-Naturalist-FormulairesJPM2019040720190602ff.xlsx'

In [ ]:
fn = f'tmp/NaturalistTestTrace-JardinJPM20190523ff.xlsx'

In [ ]:
dfObs = pd.read_excel(fn)

# Uniquement les données des formulaires
dfObs.drop(dfObs[dfObs['ID liste'] == 0].index, inplace=True) 

# Uniquement ceux d'ACDC.
dfObs.drop(dfObs[dfObs['Commentaire de la liste'].isnull()].index, inplace=True)
dfObs.drop(dfObs[~dfObs['Commentaire de la liste'].str.contains('ACDC', case=False)].index, inplace=True)

# Uniquement ceux avec trace
dfObs.drop(dfObs[dfObs['Trace'].isnull()].index, inplace=True)

dfObs[['Ref', 'Date', 'Horaire', 'Nom espèce',
       'ID liste', 'Liste complète ?', 'Commentaire de la liste',
       'Lieu-dit', 'Commune', 'Lat (WGS84)', 'Lon (WGS84)',
       'Estimation', 'Nombre', 'Détails', 'Code atlas',
       'Remarque', 'Remarque privée', 'Prénom', 'Nom',
       'Protocole', 'Trace']]

# Examen des données

In [ ]:
# Normalement, 1 trace par formulaire
dfObs['Trace'].unique()

In [ ]:
dfObs[['ID liste', 'Trace']].groupby('ID liste').nunique()

In [ ]:
# Vérifier que la trace de la liste est présente à l'identique dans toutes les données de la liste
assert all(dfObs[['ID liste', 'Trace']].groupby('ID liste').nunique().Trace == 1)

In [ ]:
# Les formulaires à traiter
dfObs[['ID liste', 'Liste complète ?', 'Commentaire de la liste', 'Trace']].groupby('ID liste').first()

In [ ]:
# Les formulaires à traiter : nbre de données
dfObs[['ID liste', 'Liste complète ?', 'Commentaire de la liste', 'Trace']].groupby('ID liste').count().Trace

# Extraction et décodage des traces

In [ ]:
# On peut donc récupérer trace de chaque liste et en extraire les points individuels
formIndCols = ['ID liste', 'Liste complète ?', 'Commentaire de la liste']
dfTraces = dfObs[formIndCols + ['Trace']].groupby(formIndCols).first()
dfTraces

In [ ]:
# Exemple de trace
dfTraces.iloc[0].Trace

In [ ]:
def decoderTrace(trace):
    return [[float(num) for num in xy.strip().split(' ')] for xy in trace[len('LINESTRING('):-len(')')].split(',')]
    
dfTraces.Trace = dfTraces.Trace.apply(decoderTrace)
dfTraces = dfTraces.Trace.apply(pd.Series).stack().reset_index()
dfTraces[['obseur_lon', 'obseur_lat']] = dfTraces.loc[:, 0].apply(pd.Series)
dfTraces.drop(columns=[0], inplace=True)
dfTraces.rename(columns=dict(level_3='NumPt'), inplace=True)
dfTraces

# Tracé cartographique des données d'un formulaire et de sa trace

In [ ]:
idListe = 440133

In [ ]:
dfTraceListe = dfTraces.loc[dfTraces['ID liste'] == idListe].copy()
dfTraceListe

In [ ]:
dfObsListe = dfObs.loc[dfObs['ID liste'] == idListe,
                       ['Lon (WGS84)', 'Lat (WGS84)', 'Date', 'Horaire', 'Nom espèce', 'Nombre', 'Code atlas']]
dfObsListe

In [ ]:
dfTraceListe['obseur_lon_sfd'] = dfTraceListe.obseur_lon.shift(-1)
dfTraceListe['obseur_lat_sfd'] = dfTraceListe.obseur_lat.shift(-1)
dfTraceListe

In [ ]:
#tiles, attr = ('http://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
#               '<a href="https://opentopomap.org/">OpenTopoMap</a> '
#               '(<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)') # OK
tiles, attr = 'http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'osm'# OK
#tiles, attr = 'https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png', 'thunderforest' # OK
mp = folium.Map(tiles=tiles, attr=attr, max_zoom=22)

# La trace
# a. Le lignes reliant les points
dfTraceListe['obseur_lon_sfd'] = dfTraceListe.obseur_lon.shift(-1)
dfTraceListe['obseur_lat_sfd'] = dfTraceListe.obseur_lat.shift(-1)
dfTraceListe.dropna(inplace=True)

commListe = dfTraceListe.iloc[0]['Commentaire de la liste']
compListe = dfTraceListe.iloc[0]['Liste complète ?']
lines = list(zip(zip(dfTraceListe.obseur_lat, dfTraceListe.obseur_lon),
                 zip(dfTraceListe.obseur_lat_sfd, dfTraceListe.obseur_lon_sfd)))[:-1]
pline = folium.PolyLine(lines, color='blue', weight=4, opacity=0.6,
                        popup=folium.Popup('#{} {} ({}complète)'.format(idListe, commListe, '' if compListe else 'in')))
pline.add_to(mp)

# b. Les points
for _, sPt in dfTraceListe.iterrows():
    mrk = folium.CircleMarker(location=(sPt.obseur_lat, sPt.obseur_lon), 
                              popup=folium.Popup('#{}: {}'.format(sPt['ID liste'], sPt.NumPt)),
                              radius=4, color='red', fill=True)
    mrk.add_to(mp)

# Les données
for indObs, sObs in dfObsListe.iterrows():
    mrk = folium.CircleMarker(location=(sObs['Lat (WGS84)'], sObs['Lon (WGS84)']), 
                              radius=8, color='orange', fill=True,
                              popup=folium.Popup('#{} {} {} {} {} (code {})' \
                                                 .format(indObs, sObs.Date, sObs.Horaire,
                                                         sObs.Nombre, sObs['Nom espèce'], sObs['Code atlas'])))
    mrk.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

# Essai via forma JSON / XML

(avant de savoir qu'elles n'y sont sont pas, le 25/01/2020)

In [ ]:
#src = 'fa'
src = 'ff'

In [ ]:
fn = f'tmp/ACDC2019-Naturalist-FormulairesJPM2019040720190602{src}.json'

In [ ]:
fn = f'tmp/NaturalistTestTrace-JardinJPM20190523{src}.json'

In [ ]:
fn = 'tmp/NaturalistTestTrace-FormRomainsAutHiv201920fa.json'

In [ ]:
dObsTr = json.load(open(fn))
type(dObsTr), type(dObsTr['data']), dObsTr['data'].keys(), type(dObsTr['data']['forms']), type(dObsTr['data']['sightings'])

In [ ]:
dict(nbFormulaires=len(dObsTr['data']['forms']), nbObsHorsFormulaires=len(dObsTr['data']['sightings']))

In [ ]:
#dObsTr

In [ ]:
#dObsTr['data']['forms'][8]

In [ ]:
for dForm in dObsTr['data']['forms']:
    print(dForm['@id'], ':', dForm['time_start'], dForm['time_stop'], dForm['lat'], dForm['lon'],
                             dForm.get('comment', ''), '=>', len(dForm['sightings']))

In [ ]:
def flattenForm(form):
    flat = odict()
    for k, v in form.items():
        if k == 'sightings':
            continue
        if isinstance(v, dict):
            for sk, sv in v.items():
                if k != 'protocol' or sk == 'protocol_name':
                    flat.update(**{ 'form_'+k+'_'+sk: sv})
        else:
            flat.update(**{ 'form_'+k: v})
    return flat

dfForms = pd.DataFrame(data=[flattenForm(form) for form in dObsTr['data']['forms']])
dfForms.set_index('form_@id', inplace=True)
dfForms

In [ ]:
def flattenSight(sight, formId):
    flat = odict([('form_@id', formId)])
    for k, v in sight.items():
        if isinstance(v, list):
            v = v[0]
        for sk, sv in v.items():
            if isinstance(sv, dict):
                for ssk, ssv in sv.items():
                    flat.update(**{ k+'_'+sk+'_'+ssk: ssv})
            else:
                flat.update(**{ k+'_'+sk: sv})
    return flat

ldfSights = list()
for form in dObsTr['data']['forms']:
    dfSights = pd.DataFrame(data=[flattenSight(sight, form['@id']) for sight in form['sightings']])
    dfSights = dfSights.join(dfForms, on=['form_@id'])
    ldfSights.append(dfSights)
    
dfSights = pd.concat(ldfSights, sort=False, ignore_index=True)
dfSights

In [ ]:
dfSights.columns

In [ ]:
if 'form_protocol_protocol_name' in dfSights.columns:
    dfSights.drop(dfSights[dfSights['form_protocol_protocol_name'] == 'STOC_EPS'].index, inplace=True)
dfSights['form_comment'].fillna('', inplace=True)
dfSights.drop(dfSights[~dfSights['form_comment'].str.contains('ACDC', case=False)].index, inplace=True)

dfSights

In [ ]:
dfSights['date_@ISO8601'].min(), dfSights['date_@ISO8601'].max()

In [ ]:
dfSights[['form_@id']+[col for col in dfSights.columns if col.endswith('lat') or col.endswith('lon')]].head(30)